In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('../data/regionali-20200920/regionali-20200920.xlsx')

In [3]:
df.head(2)

,DATAELEZIONE,ELETTORI_TOT,ELETTORI_M,VOTANTI_TOT,VOTANTI_M,SKBIANCHE,DESCRLISTA,VOTILISTA,COGNOME,NOME,DATANASCITA,LUOGONASCITA,SESSO,CODTIPOELETTO,VOTICAND,REGIONE,CIRCOSCR,PROVINCIA,COMUNE
0,2020-09-20,3861,1844,2030,984,21,VENETO PER LE AUTONOMIE,12.0,RUBINATO,SIMONETTA,1963-12-22,TREVISO,F,NaN,18,VENETO,BELLUNO,BELLUNO,AGORDO
1,2020-09-20,3861,1844,2030,984,21,ZAIA PRESIDENTE,839.0,ZAIA,LUCA,1968-03-27,CONEGLIANO TV,M,S,1514,VENETO,BELLUNO,BELLUNO,AGORDO


In [4]:
# Group by comune and candidate to get total votes
votes_by_candidate = df.groupby(['COMUNE', 'COGNOME', 'NOME'])['VOTICAND'].first().reset_index()

# Filter for only Toti and Sansa
votes_by_candidate = votes_by_candidate[votes_by_candidate['COGNOME'].isin(['TOTI', 'SANSA'])]

# Create candidate full names
votes_by_candidate['candidate_full'] = votes_by_candidate['NOME'] + ' ' + votes_by_candidate['COGNOME']

# Pivot the data to get one row per comune with columns for each candidate
pivot_df = votes_by_candidate.pivot(
    index='COMUNE',
    columns='candidate_full',
    values='VOTICAND'
).reset_index()

# Get total voters and blank ballots by comune and merge with pivot_df
voters_info = df.groupby('COMUNE').agg({
    'VOTANTI_TOT': 'first',
    'SKBIANCHE': 'first'
}).reset_index()

# Calculate valid votes (total votes minus blank ballots)
voters_info['VOTI_VALIDI'] = voters_info['VOTANTI_TOT'] - voters_info['SKBIANCHE']

# Merge with pivot_df
pivot_df = pd.merge(pivot_df, voters_info, on='COMUNE')

# Calculate percentages
for candidate in ['GIOVANNI TOTI', 'FERRUCCIO SANSA']:
    if candidate in pivot_df.columns:
        pivot_df[f'{candidate}_total_perc'] = (pivot_df[candidate] / pivot_df['VOTI_VALIDI'] * 100).round(2)
        # Rename votes column to match desired format
        pivot_df = pivot_df.rename(columns={candidate: f'{candidate}_total_votes'})

# Select and order columns as needed
final_columns = [
    'COMUNE',
    'VOTANTI_TOT',
    'SKBIANCHE',
    'VOTI_VALIDI',
    'GIOVANNI TOTI_total_votes',
    'GIOVANNI TOTI_total_perc',
    'FERRUCCIO SANSA_total_votes',
    'FERRUCCIO SANSA_total_perc'
]

clean_df = pivot_df[final_columns]

# Rename COMUNE to comune for consistency
clean_df = clean_df.rename(columns={'COMUNE': 'comune'})

# Display first few rows to verify
print(clean_df.head())

               comune  VOTANTI_TOT  SKBIANCHE  VOTI_VALIDI  \
0              AIROLE          148          3          145   
1             ALASSIO         4831         81         4750   
2             ALBENGA        11013        269        10744   
3  ALBISOLA SUPERIORE         4621         99         4522   
4    ALBISSOLA MARINA         2989         72         2917   

   GIOVANNI TOTI_total_votes  GIOVANNI TOTI_total_perc  \
0                         70                     48.28   
1                       2985                     62.84   
2                       6839                     63.65   
3                       2353                     52.03   
4                       1514                     51.90   

   FERRUCCIO SANSA_total_votes  FERRUCCIO SANSA_total_perc  
0                           53                       36.55  
1                         1496                       31.49  
2                         3045                       28.34  
3                         1741    

In [5]:
clean_df.head(20)

,comune,VOTANTI_TOT,SKBIANCHE,VOTI_VALIDI,GIOVANNI TOTI_total_votes,GIOVANNI TOTI_total_perc,FERRUCCIO SANSA_total_votes,FERRUCCIO SANSA_total_perc
0,AIROLE,148,3,145,70,48.28,53,36.55
1,ALASSIO,4831,81,4750,2985,62.84,1496,31.49
2,ALBENGA,11013,269,10744,6839,63.65,3045,28.34
3,ALBISOLA SUPERIORE,4621,99,4522,2353,52.03,1741,38.50
4,ALBISSOLA MARINA,2989,72,2917,1514,51.90,1198,41.07
5,ALTARE,909,22,887,476,53.66,339,38.22
6,AMEGLIA,2086,37,2049,1210,59.05,666,32.50
7,ANDORA,3749,83,3666,2646,72.18,770,21.00
8,APRICALE,217,8,209,100,47.85,81,38.76
9,AQUILA D'ARROSCIA,123,8,115,100,86.96,10,8.70


In [6]:
# Save to CSV
clean_df.to_csv('../output/risultati_li_2020.csv', index=False)

## Now mergin with the new data

In [7]:
df = pd.read_csv('../output/risultati_li_2020.csv')
df.columns

Index(['comune', 'VOTANTI_TOT', 'SKBIANCHE', 'VOTI_VALIDI',
       'GIOVANNI TOTI_total_votes', 'GIOVANNI TOTI_total_perc',
       'FERRUCCIO SANSA_total_votes', 'FERRUCCIO SANSA_total_perc'],
      dtype='object')

In [8]:
# Select and rename the columns we want
clean_df = df[['comune', 
               'GIOVANNI TOTI_total_votes', 'GIOVANNI TOTI_total_perc',
               'FERRUCCIO SANSA_total_votes', 'FERRUCCIO SANSA_total_perc']]

# Rename columns to make them cleaner
clean_df = clean_df.rename(columns={
    'GIOVANNI TOTI_total_votes': 'CDX_votes_2020',
    'GIOVANNI TOTI_total_perc': 'CDX_perc_2020',
    'FERRUCCIO SANSA_total_votes': 'CSX_votes_2020',
    'FERRUCCIO SANSA_total_perc': 'CSX_perc_2020'
})

# Display the first few rows of the cleaned dataset
print("\nCleaned data preview:")
print(clean_df.head())

# Display some basic statistics
print("\nBasic statistics:")
print(clean_df.describe())

clean_df['comune'] = clean_df['comune'].str.strip().str.title()


Cleaned data preview:
               comune  CDX_votes_2020  CDX_perc_2020  CSX_votes_2020  \
0              AIROLE              70          48.28              53   
1             ALASSIO            2985          62.84            1496   
2             ALBENGA            6839          63.65            3045   
3  ALBISOLA SUPERIORE            2353          52.03            1741   
4    ALBISSOLA MARINA            1514          51.90            1198   

   CSX_perc_2020  
0          36.55  
1          31.49  
2          28.34  
3          38.50  
4          41.07  

Basic statistics:
       CDX_votes_2020  CDX_perc_2020  CSX_votes_2020  CSX_perc_2020
count      234.000000     234.000000      234.000000     234.000000
mean      1636.978632      60.490598     1134.641026      30.925299
std       8600.920433       9.489674     7175.475297       9.035186
min         19.000000      30.990000        8.000000       8.700000
25%        177.000000      53.932500       71.750000      24.767500
50%

In [9]:
df2 = pd.read_csv('../output/risultati_li.csv')
df2

,comune,provincia,regione,url,PARTITO DEMOCRATICO ANDREA ORLANDO PRESIDENTE,PARTITO DEMOCRATICO ANDREA ORLANDO PRESIDENTE_perc,LISTA ANDREA ORLANDO PRESIDENTE,LISTA ANDREA ORLANDO PRESIDENTE_perc,ALLEANZA VERDI E SINISTRA - LISTA SANSA - POSSIBILE - EUROPA VERDE - SINISTRA ITALIANA,ALLEANZA VERDI E SINISTRA - LISTA SANSA - POSSIBILE - EUROPA VERDE - SINISTRA ITALIANA_perc,...,DAVIDE FELICE_total_votes,DAVIDE FELICE_total_perc,MARCO GIUSEPPE FERRANDO_total_votes,MARCO GIUSEPPE FERRANDO_total_perc,ALESSANDRO ROSSON_total_votes,ALESSANDRO ROSSON_total_perc,MARIA ANTONIETTA CELLA_total_votes,MARIA ANTONIETTA CELLA_total_perc,Unnamed: 62,None_perc
0,Arenzano,Genova,Liguria,https://elezioni.interno.gov.it/risultati/2024...,1529,"28,56",677,"12,65",239,"4,46",...,16,"0,28",16,"0,28",14,"0,25",3,"0,05",NaN,NaN
1,Avegno,Genova,Liguria,https://elezioni.interno.gov.it/risultati/2024...,205,"20,16",40,"3,93",47,"4,62",...,0,0,6,"0,56",2,"0,19",5,"0,47",NaN,NaN
2,Bargagli,Genova,Liguria,https://elezioni.interno.gov.it/risultati/2024...,219,"24,72",73,"8,24",36,"4,06",...,4,"0,42",5,"0,53",2,"0,21",2,"0,21",NaN,NaN
3,Bogliasco,Genova,Liguria,https://elezioni.interno.gov.it/risultati/2024...,610,"30,32",152,"7,55",89,"4,42",...,6,"0,28",7,"0,33",1,"0,05",12,"0,56",NaN,NaN
4,Borzonasca,Genova,Liguria,https://elezioni.interno.gov.it/risultati/2024...,86,"15,22",27,"4,78",23,"4,07",...,2,"0,34",2,"0,34",1,"0,17",50,"8,40",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Varazze,Savona,Liguria,https://elezioni.interno.gov.it/risultati/2024...,1181,"23,35",150,"2,97",207,"4,09",...,40,"0,74",28,"0,51",14,"0,26",14,"0,26",NaN,0.0
230,Vendone,Savona,Liguria,https://elezioni.interno.gov.it/risultati/2024...,36,"24,32",7,"4,73",8,"5,41",...,0,0,1,"0,65",1,"0,65",0,0,NaN,0.0
231,Vezzi Portio,Savona,Liguria,https://elezioni.interno.gov.it/risultati/2024...,110,"37,80",6,"2,06",25,"8,59",...,0,0,2,"0,65",2,"0,65",1,"0,33",NaN,0.0
232,Villanova D'Albenga,Savona,Liguria,https://elezioni.interno.gov.it/risultati/2024...,250,"24,75",44,"4,36",64,"6,34",...,8,"0,74",6,"0,56",6,"0,56",2,"0,19",NaN,0.0


In [10]:
df2 = pd.read_csv('../output/risultati_li.csv')

# Print original columns to verify we have the right candidate columns
print("Original columns:")
print(df.columns.tolist())

# Select and rename the columns we want
clean_df2 = df2[['comune', 'provincia', 'regione',
               'ANDREA ORLANDO_total_votes', 'ANDREA ORLANDO_total_perc',
               'MARCO BUCCI_total_votes', 'MARCO BUCCI_total_perc']]

# Rename columns to make them cleaner
clean_df2 = clean_df2.rename(columns={
    'ANDREA ORLANDO_total_votes': 'CSX_votes_2024',
    'ANDREA ORLANDO_total_perc': 'CSX_perc_2024',
    'MARCO BUCCI_total_votes': 'CDX_votes_2024',
    'MARCO BUCCI_total_perc': 'CDX_perc_2024'
})

# Display the first few rows of the cleaned dataset
print("\nCleaned data preview:")
print(clean_df2.head())

clean_df2.sample(5)

Original columns:
['comune', 'VOTANTI_TOT', 'SKBIANCHE', 'VOTI_VALIDI', 'GIOVANNI TOTI_total_votes', 'GIOVANNI TOTI_total_perc', 'FERRUCCIO SANSA_total_votes', 'FERRUCCIO SANSA_total_perc']

Cleaned data preview:
       comune provincia  regione  CSX_votes_2024 CSX_perc_2024  \
0    Arenzano    Genova  Liguria            2911         51,02   
1      Avegno    Genova  Liguria             368         34,42   
2    Bargagli    Genova  Liguria             442         46,92   
3   Bogliasco    Genova  Liguria            1035         48,52   
4  Borzonasca    Genova  Liguria             191         32,10   

   CDX_votes_2024 CDX_perc_2024  
0            2647         46,39  
1             661         61,83  
2             458         48,62  
3            1029         48,24  
4             334         56,13  


,comune,provincia,regione,CSX_votes_2024,CSX_perc_2024,CDX_votes_2024,CDX_perc_2024
82,Cervo,Imperia,Liguria,136,"40,48",187,"55,65"
28,Leivi,Genova,Liguria,387,"36,65",620,"58,71"
14,Chiavari,Genova,Liguria,4573,"39,24",6529,"56,02"
182,Carcare,Savona,Liguria,1095,"55,36",799,"40,39"
155,Ricco' Del Golfo Di Spezia,La Spezia,Liguria,719,"49,76",682,"47,20"


In [11]:
merged_df = pd.merge(clean_df, clean_df2, on='comune', how='inner')
merged_df

,comune,CDX_votes_2020,CDX_perc_2020,CSX_votes_2020,CSX_perc_2020,provincia,regione,CSX_votes_2024,CSX_perc_2024,CDX_votes_2024,CDX_perc_2024
0,Airole,70,48.28,53,36.55,Imperia,Liguria,72,"49,32",67,"45,89"
1,Alassio,2985,62.84,1496,31.49,Savona,Liguria,1554,"40,82",2141,"56,24"
2,Albenga,6839,63.65,3045,28.34,Savona,Liguria,3891,"44,70",4544,"52,21"
3,Albisola Superiore,2353,52.03,1741,38.50,Savona,Liguria,2053,"49,43",1922,"46,28"
4,Albissola Marina,1514,51.90,1198,41.07,Savona,Liguria,1245,"50,69",1123,"45,72"
...,...,...,...,...,...,...,...,...,...,...,...
229,Villanova D'Albenga,1092,74.24,286,19.44,Savona,Liguria,404,"37,51",629,"58,40"
230,Vobbia,136,53.12,68,26.56,Genova,Liguria,71,"41,04",96,"55,49"
231,Zignago,200,68.97,76,26.21,La Spezia,Liguria,75,"32,75",146,"63,76"
232,Zoagli,926,64.85,430,30.11,Genova,Liguria,311,"32,40",588,"61,25"


In [12]:
merged_df['CSX_perc_2024'] = merged_df['CSX_perc_2024'].str.replace(',','.').astype(float)
merged_df['CDX_perc_2024'] = merged_df['CDX_perc_2024'].str.replace(',','.').astype(float)

merged_df.sample(5)

,comune,CDX_votes_2020,CDX_perc_2020,CSX_votes_2020,CSX_perc_2020,provincia,regione,CSX_votes_2024,CSX_perc_2024,CDX_votes_2024,CDX_perc_2024
103,Isolabona,188,65.28,83,28.82,Imperia,Liguria,88,38.77,131,57.71
189,Santo Stefano D'Aveto,362,79.91,64,14.13,Genova,Liguria,71,15.88,229,51.23
3,Albisola Superiore,2353,52.03,1741,38.50,Savona,Liguria,2053,49.43,1922,46.28
70,Cipressa,344,63.94,139,25.84,Imperia,Liguria,179,41.15,234,53.79
56,Castelnuovo Magra,1839,47.75,1717,44.59,La Spezia,Liguria,1622,52.00,1365,43.76


In [13]:
merged_df['margin_2020'] = merged_df['CSX_perc_2020'] - merged_df['CDX_perc_2020']
merged_df['margin_2024'] = merged_df['CSX_perc_2024'] - merged_df['CDX_perc_2024']
merged_df['margin_shift'] = merged_df['margin_2024'] - merged_df['margin_2020']

GEOLOCATING

In [14]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time

In [15]:
# Create the geocoder
geolocator = Nominatim(user_agent='my_geocoder')
# Create a delay between requests to respect usage limits
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_location(row):
    try:
        # Create a complete address string
        address = f"{row['comune']}, {row['provincia']}, {row['regione']}, Italy"
        location = geocode(address)
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Error geocoding {row['comune']}: {str(e)}")
        return pd.Series([None, None])

In [16]:
# Add coordinates
print("Starting geocoding process...")
merged_df[['latitude', 'longitude']] = merged_df.apply(get_location, axis=1)

# Check for any missing coordinates
missing_coords = merged_df[merged_df['latitude'].isna() | merged_df['longitude'].isna()]
if not missing_coords.empty:
    print("\nWarning: Could not find coordinates for these comuni:")
    print(missing_coords[['comune', 'provincia']])

# Display sample of results
print("\nSample of geocoded results:")
print(merged_df[['comune', 'provincia', 'latitude', 'longitude']].head())


Starting geocoding process...

Sample of geocoded results:
               comune provincia   latitude  longitude
0              Airole   Imperia  43.871038   7.554181
1             Alassio    Savona  44.008011   8.173027
2             Albenga    Savona  44.049250   8.213124
3  Albisola Superiore    Savona  44.339838   8.509353
4    Albissola Marina    Savona  44.327272   8.503189


In [17]:
merged_df['margin_2020'] = merged_df['CDX_perc_2020'] - merged_df['CSX_perc_2020']
merged_df['margin_2024'] = merged_df['CDX_perc_2024'] - merged_df['CSX_perc_2024']
merged_df['margin_shift'] = merged_df['margin_2024'] - merged_df['margin_2020']

In [18]:
merged_df.to_csv('../output/viz/margini_elettorali_LI.csv', index=False, encoding='UTF-8')